# Importing libraries and processing the data

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

from power.ml_ops.cross_val import get_folds, train_test_split, get_X_y_seq
from power.ml_ops.model import init_baseline_keras, compile_model, initialize_model, train_model

# tensforflow
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.layers import Lambda

pd.set_option('display.max_rows', 500)

2024-03-13 17:31:28.565732: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-13 17:31:30.032358: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-13 17:31:30.038963: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 17:31:36.551628: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
raw_data = pd.read_csv('../raw_data/history_forecast_bulk_20171007_20240312.csv')

In [3]:
def compress(df, **kwargs):
    """
    Reduces size of dataframe by downcasting numerical columns
    """
    input_size = df.memory_usage(index=True).sum()/ 1024
    print("new dataframe size: ", round(input_size,2), 'kB')

    in_size = df.memory_usage(index=True).sum()
    for type in ["float", "integer"]:
        l_cols = list(df.select_dtypes(include=type))
        for col in l_cols:
            df[col] = pd.to_numeric(df[col], downcast=type)
    out_size = df.memory_usage(index=True).sum()
    ratio = (1 - round(out_size / in_size, 2)) * 100

    print("optimized size by {} %".format(round(ratio,2)))
    print("new dataframe size: ", round(out_size / 1024,2), " kB")

    return df

In [4]:
raw_data = compress(raw_data)

new dataframe size:  617298.69 kB
optimized size by 46.0 %
new dataframe size:  334370.18  kB


In [7]:
df = raw_data.copy()

In [9]:
df = df[['forecast dt iso', 'slice dt iso', 'temperature', 'dew_point', 'pressure',
       'ground_pressure', 'humidity', 'clouds', 'wind_speed', 'wind_deg',
       'rain', 'snow', 'ice', 'fr_rain', 'convective', 'snow_depth',
       'accumulated', 'hours', 'rate', 'probability']]

In [10]:
df['forecast dt iso'] = df['forecast dt iso'].str.replace('+0000 UTC', '')
df['slice dt iso'] = df['slice dt iso'].str.replace('+0000 UTC', '')


In [11]:
df.head()

,forecast dt iso,slice dt iso,temperature,dew_point,pressure,ground_pressure,humidity,clouds,wind_speed,wind_deg,rain,snow,ice,fr_rain,convective,snow_depth,accumulated,hours,rate,probability
0,2017-10-07 00:00:00,2017-10-07 00:00:00,9.07,7.75,1015.809998,1010.159973,91.800003,0.0,3.78,278.109985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.00
1,2017-10-07 00:00:00,2017-10-07 01:00:00,9.49,7.62,1015.750000,1010.190002,88.919998,33.0,4.64,280.850006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.04
2,2017-10-07 00:00:00,2017-10-07 02:00:00,9.64,7.68,1015.799988,1010.289978,88.410004,54.0,5.09,279.799988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.03
3,2017-10-07 00:00:00,2017-10-07 03:00:00,9.63,7.85,1015.909973,1010.400024,89.110001,64.0,5.25,276.329987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.00
4,2017-10-07 00:00:00,2017-10-07 04:00:00,9.61,8.02,1016.030029,1010.520020,89.870003,68.0,5.26,271.809998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000008,0.00


In [12]:
df = df[df['forecast dt iso'].str.contains('12:00:00')]
df

,forecast dt iso,slice dt iso,temperature,dew_point,pressure,ground_pressure,humidity,clouds,wind_speed,wind_deg,rain,snow,ice,fr_rain,convective,snow_depth,accumulated,hours,rate,probability
506,2017-10-07 12:00:00,2017-10-07 12:00:00,13.35,5.65,1013.489990,1008.030029,59.459999,0.0,6.08,256.480011,0.00,0.0,0.0,0.0,0.000,0.0,0.00,1.0,0.000000,0.00
507,2017-10-07 12:00:00,2017-10-07 13:00:00,13.04,5.96,1012.700012,1007.210022,61.860001,52.0,5.80,242.679993,0.00,0.0,0.0,0.0,0.000,0.0,0.00,1.0,0.000025,0.37
508,2017-10-07 12:00:00,2017-10-07 14:00:00,12.50,6.18,1011.849976,1006.359985,65.199997,84.0,5.99,234.029999,0.18,0.0,0.0,0.0,0.000,0.0,0.18,1.0,0.000050,0.64
509,2017-10-07 12:00:00,2017-10-07 15:00:00,11.85,6.37,1010.940002,1005.479980,69.070000,100.0,6.46,229.570007,0.27,0.0,0.0,0.0,0.021,0.0,0.27,1.0,0.000075,0.81
510,2017-10-07 12:00:00,2017-10-07 16:00:00,11.19,6.55,1009.979980,1004.559998,73.040001,100.0,7.05,228.320007,0.36,0.0,0.0,0.0,0.021,0.0,0.36,1.0,0.000100,0.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3291869,2024-03-12 12:00:00,2024-03-28 08:00:00,8.98,3.33,1017.969971,1012.099976,68.169998,30.0,6.24,240.039993,0.00,0.0,0.0,0.0,2.958,0.0,0.00,1.0,0.000000,0.01
3291870,2024-03-12 12:00:00,2024-03-28 09:00:00,10.24,3.51,1018.580017,1012.719971,63.000000,20.0,6.75,241.460007,0.00,0.0,0.0,0.0,2.958,0.0,0.00,1.0,0.000000,0.00
3291871,2024-03-12 12:00:00,2024-03-28 10:00:00,11.08,3.69,1019.030029,1013.179993,59.759998,19.0,7.03,245.419998,0.00,0.0,0.0,0.0,2.958,0.0,0.00,1.0,0.000017,0.01
3291872,2024-03-12 12:00:00,2024-03-28 11:00:00,11.08,3.80,1019.260010,1013.409973,60.160000,30.0,6.91,252.869995,0.14,0.0,0.0,0.0,2.958,0.0,0.14,1.0,0.000036,0.07


In [13]:
df['forecast dt iso'] = pd.to_datetime(df['forecast dt iso'])
df['slice dt iso'] = pd.to_datetime(df['slice dt iso'])

In [14]:
df.head()

,forecast dt iso,slice dt iso,temperature,dew_point,pressure,ground_pressure,humidity,clouds,wind_speed,wind_deg,rain,snow,ice,fr_rain,convective,snow_depth,accumulated,hours,rate,probability
506,2017-10-07 12:00:00,2017-10-07 12:00:00,13.35,5.65,1013.489990,1008.030029,59.459999,0.0,6.08,256.480011,0.00,0.0,0.0,0.0,0.000,0.0,0.00,1.0,0.000000,0.00
507,2017-10-07 12:00:00,2017-10-07 13:00:00,13.04,5.96,1012.700012,1007.210022,61.860001,52.0,5.80,242.679993,0.00,0.0,0.0,0.0,0.000,0.0,0.00,1.0,0.000025,0.37
508,2017-10-07 12:00:00,2017-10-07 14:00:00,12.50,6.18,1011.849976,1006.359985,65.199997,84.0,5.99,234.029999,0.18,0.0,0.0,0.0,0.000,0.0,0.18,1.0,0.000050,0.64
509,2017-10-07 12:00:00,2017-10-07 15:00:00,11.85,6.37,1010.940002,1005.479980,69.070000,100.0,6.46,229.570007,0.27,0.0,0.0,0.0,0.021,0.0,0.27,1.0,0.000075,0.81
510,2017-10-07 12:00:00,2017-10-07 16:00:00,11.19,6.55,1009.979980,1004.559998,73.040001,100.0,7.05,228.320007,0.36,0.0,0.0,0.0,0.021,0.0,0.36,1.0,0.000100,0.92


In [15]:
df_unique_dates = df['forecast dt iso'].unique()

In [16]:
df[(df['forecast dt iso'] == df_unique_dates[0]) & (df['slice dt iso'].between(df_unique_dates[0] + timedelta(days=1) - timedelta(hours=12), df_unique_dates[0] + timedelta(days=1) + timedelta(hours=11)))]

,forecast dt iso,slice dt iso,temperature,dew_point,pressure,ground_pressure,humidity,clouds,wind_speed,wind_deg,rain,snow,ice,fr_rain,convective,snow_depth,accumulated,hours,rate,probability
518,2017-10-07 12:00:00,2017-10-08 00:00:00,11.32,8.95,1004.539978,999.000000,85.199997,100.0,7.20,256.970001,0.33,0.0,0.0,0.0,0.000,0.0,0.33,1.0,0.000092,1.00
519,2017-10-07 12:00:00,2017-10-08 01:00:00,11.61,9.33,1004.570007,998.989990,85.790001,100.0,6.63,264.929993,0.24,0.0,0.0,0.0,0.000,0.0,0.24,1.0,0.000067,0.93
520,2017-10-07 12:00:00,2017-10-08 02:00:00,11.88,9.72,1004.760010,999.140015,86.470001,100.0,5.95,272.799988,0.19,0.0,0.0,0.0,0.000,0.0,0.19,1.0,0.000053,0.83
521,2017-10-07 12:00:00,2017-10-08 03:00:00,11.96,9.95,1005.140015,999.469971,87.400002,99.0,5.53,278.609985,0.15,0.0,0.0,0.0,0.042,0.0,0.15,1.0,0.000042,0.73
522,2017-10-07 12:00:00,2017-10-08 04:00:00,11.73,9.91,1005.750000,1000.030029,88.480003,98.0,5.63,281.290009,0.11,0.0,0.0,0.0,0.042,0.0,0.11,1.0,0.000031,0.67
523,2017-10-07 12:00:00,2017-10-08 05:00:00,11.40,9.61,1006.510010,1000.750000,88.669998,95.0,5.98,283.230011,0.00,0.0,0.0,0.0,0.042,0.0,0.00,1.0,0.000025,0.65
524,2017-10-07 12:00:00,2017-10-08 06:00:00,11.25,9.09,1007.299988,1001.559998,86.720001,91.0,6.21,287.690002,0.00,0.0,0.0,0.0,0.083,0.0,0.00,1.0,0.000022,0.67
525,2017-10-07 12:00:00,2017-10-08 07:00:00,11.48,8.41,1008.070007,1002.409973,81.800003,84.0,6.06,296.670013,0.10,0.0,0.0,0.0,0.083,0.0,0.10,1.0,0.000028,0.72
526,2017-10-07 12:00:00,2017-10-08 08:00:00,11.97,7.62,1008.799988,1003.250000,74.980003,76.0,5.66,307.140015,0.13,0.0,0.0,0.0,0.083,0.0,0.13,1.0,0.000036,0.78
527,2017-10-07 12:00:00,2017-10-08 09:00:00,12.56,6.75,1009.539978,1004.070007,67.800003,67.0,5.28,314.850006,0.15,0.0,0.0,0.0,0.146,0.0,0.15,1.0,0.000042,0.82


In [17]:
df_revised = []
for date in df_unique_dates:
    data = df[(df['forecast dt iso'] == date) & (df['slice dt iso'].between(date + timedelta(days=1) - timedelta(hours=12), date + timedelta(days=1) + timedelta(hours=11)))]
    df_revised.append(data)

In [18]:
df_revised_ordered = pd.concat(df_revised, ignore_index=True)

In [19]:
df_revised_ordered

,forecast dt iso,slice dt iso,temperature,dew_point,pressure,ground_pressure,humidity,clouds,wind_speed,wind_deg,rain,snow,ice,fr_rain,convective,snow_depth,accumulated,hours,rate,probability
0,2017-10-07 12:00:00,2017-10-08 00:00:00,11.32,8.95,1004.539978,999.000000,85.199997,100.0,7.20,256.970001,0.33,0.0,0.0,0.0,0.000,0.0,0.33,1.0,0.000092,1.00
1,2017-10-07 12:00:00,2017-10-08 01:00:00,11.61,9.33,1004.570007,998.989990,85.790001,100.0,6.63,264.929993,0.24,0.0,0.0,0.0,0.000,0.0,0.24,1.0,0.000067,0.93
2,2017-10-07 12:00:00,2017-10-08 02:00:00,11.88,9.72,1004.760010,999.140015,86.470001,100.0,5.95,272.799988,0.19,0.0,0.0,0.0,0.000,0.0,0.19,1.0,0.000053,0.83
3,2017-10-07 12:00:00,2017-10-08 03:00:00,11.96,9.95,1005.140015,999.469971,87.400002,99.0,5.53,278.609985,0.15,0.0,0.0,0.0,0.042,0.0,0.15,1.0,0.000042,0.73
4,2017-10-07 12:00:00,2017-10-08 04:00:00,11.73,9.91,1005.750000,1000.030029,88.480003,98.0,5.63,281.290009,0.11,0.0,0.0,0.0,0.042,0.0,0.11,1.0,0.000031,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56371,2024-03-12 12:00:00,2024-03-13 19:00:00,10.53,3.09,1016.969971,1011.140015,59.790001,90.0,2.42,216.630005,0.00,0.0,0.0,0.0,0.042,0.0,0.00,1.0,0.000000,0.00
56372,2024-03-12 12:00:00,2024-03-13 20:00:00,10.18,3.04,1017.039978,1011.219971,61.009998,87.0,2.39,206.800003,0.00,0.0,0.0,0.0,0.042,0.0,0.00,1.0,0.000000,0.00
56373,2024-03-12 12:00:00,2024-03-13 21:00:00,9.93,3.27,1017.039978,1011.210022,63.400002,86.0,2.41,201.100006,0.00,0.0,0.0,0.0,0.042,0.0,0.00,1.0,0.000000,0.00
56374,2024-03-12 12:00:00,2024-03-13 22:00:00,9.74,4.06,1016.960022,1011.130005,68.389999,87.0,2.45,204.210007,0.00,0.0,0.0,0.0,0.042,0.0,0.00,1.0,0.000000,0.04


In [20]:
pv_weather_df = df_revised_ordered[df_revised_ordered['slice dt iso'] <= '2022-12-31 23:00:00']

In [21]:
pv_weather_df

,forecast dt iso,slice dt iso,temperature,dew_point,pressure,ground_pressure,humidity,clouds,wind_speed,wind_deg,rain,snow,ice,fr_rain,convective,snow_depth,accumulated,hours,rate,probability
0,2017-10-07 12:00:00,2017-10-08 00:00:00,11.32,8.95,1004.539978,999.000000,85.199997,100.0,7.20,256.970001,0.33,0.0,0.0,0.0,0.000,0.0,0.33,1.0,0.000092,1.00
1,2017-10-07 12:00:00,2017-10-08 01:00:00,11.61,9.33,1004.570007,998.989990,85.790001,100.0,6.63,264.929993,0.24,0.0,0.0,0.0,0.000,0.0,0.24,1.0,0.000067,0.93
2,2017-10-07 12:00:00,2017-10-08 02:00:00,11.88,9.72,1004.760010,999.140015,86.470001,100.0,5.95,272.799988,0.19,0.0,0.0,0.0,0.000,0.0,0.19,1.0,0.000053,0.83
3,2017-10-07 12:00:00,2017-10-08 03:00:00,11.96,9.95,1005.140015,999.469971,87.400002,99.0,5.53,278.609985,0.15,0.0,0.0,0.0,0.042,0.0,0.15,1.0,0.000042,0.73
4,2017-10-07 12:00:00,2017-10-08 04:00:00,11.73,9.91,1005.750000,1000.030029,88.480003,98.0,5.63,281.290009,0.11,0.0,0.0,0.0,0.042,0.0,0.11,1.0,0.000031,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45859,2022-12-30 12:00:00,2022-12-31 19:00:00,14.22,9.86,1011.669983,1005.960022,74.970001,100.0,7.52,224.029999,0.00,0.0,0.0,0.0,0.062,0.0,0.00,1.0,0.000000,0.01
45860,2022-12-30 12:00:00,2022-12-31 20:00:00,14.13,9.57,1011.770020,1006.059998,73.919998,100.0,7.54,220.160004,0.00,0.0,0.0,0.0,0.062,0.0,0.00,1.0,0.000000,0.01
45861,2022-12-30 12:00:00,2022-12-31 21:00:00,14.09,9.18,1011.909973,1006.210022,72.199997,100.0,7.56,217.789993,0.00,0.0,0.0,0.0,0.062,0.0,0.00,1.0,0.000000,0.00
45862,2022-12-30 12:00:00,2022-12-31 22:00:00,14.07,8.75,1012.140015,1006.440002,70.220001,98.0,7.54,218.410004,0.00,0.0,0.0,0.0,0.062,0.0,0.00,1.0,0.000000,0.00


In [22]:
df = pd.read_csv('../raw_data/1980-2022_pv.csv')

In [23]:
df = df.drop(columns=['irradiance_direct', 'source', '_0-1', 'temperature', 'irradiance_diffuse', 'local_time'])
df.electricity = df.electricity.astype(float)
df['_0'] = pd.to_datetime(df['_0'], unit='ms').dt.tz_localize('UTC')
df.rename(columns={'_0': 'utc_time'}, inplace=True)
df.utc_time = pd.to_datetime(df.utc_time,utc=True)

In [24]:
df = df[df['utc_time'] >= '2017-10-08 00:00:00'].reset_index(drop=True)
df

,utc_time,electricity
0,2017-10-08 00:00:00+00:00,0.0
1,2017-10-08 01:00:00+00:00,0.0
2,2017-10-08 02:00:00+00:00,0.0
3,2017-10-08 03:00:00+00:00,0.0
4,2017-10-08 04:00:00+00:00,0.0
...,...,...
45859,2022-12-31 19:00:00+00:00,0.0
45860,2022-12-31 20:00:00+00:00,0.0
45861,2022-12-31 21:00:00+00:00,0.0
45862,2022-12-31 22:00:00+00:00,0.0


In [25]:
pv_df = df.copy()

In [ ]:
############################################# For Jerome - ignore the rest ############################################################

In [ ]:
pv_weather_df = pv_weather_df.rename(columns={'forecast dt iso': 'date_of_forcast',
                                              'slice dt iso': 'forcasting_date_range',
                                              'fr_rain': 'freezing_rain_vol',
                                              ''})

In [ ]:
pv_df

In [ ]:
comb_df = pd.merge(pv_weather_df, pv_df, )

In [ ]:
pv_df = df.copy()

In [ ]:
min_date = '1980-01-01 00:00:00'
max_date = '2019-12-31 23:00:00'
train = pv_df[pv_df['utc_time'] <= max_date]
test = pv_df[pv_df['utc_time'] > max_date]

In [ ]:
train = train[['electricity']]
test = test[['electricity']]

# Cross validation

In [ ]:
TARGET = 'electricity'
fold_length = 43800             # 5 years
fold_stride = 43800             # 5 years
train_test_ratio = 0.8          # 5 yrs/6 yrs
input_length = 48               # number of obsevations per one sequence
output_length = 24              # Day-ahead predictions
n_seq_train = 500               # number_of_sequences_train
n_seq_val = 100                 # number_of_sequences_test
n_unit = 48                     # number of hidden units
learning_rate = 0.02
patience = 5
epochs = 50
batch_size = 32

In [ ]:
def cross_validate_baseline_and_lstm():

    list_of_mae_baseline_model = []
    list_of_mae_recurrent_model = []

    # 0 - Creating folds
    # =========================================
    folds = get_folds(train, fold_length, fold_stride)

    for fold_id, fold in enumerate(folds):

        # 1 - Train/Test split the current fold
        # =========================================
        (fold_train, fold_val) = train_test_split(fold, train_test_ratio, input_length)

        X_train, y_train = get_X_y_seq(fold_train, n_seq_train, input_length, output_length, gap_hours=12)
        X_val, y_val = get_X_y_seq(fold_val, n_seq_val, input_length, output_length, gap_hours=12)

        # 2 - Modelling
        # =========================================

        ##### Baseline Model
        baseline_model = init_baseline_keras()
        mae_baseline = baseline_model.evaluate(X_val, y_val, verbose=0)[1]
        list_of_mae_baseline_model.append(mae_baseline)
        print("-"*50)
        print(f"MAE baseline fold n°{fold_id} = {round(mae_baseline, 2)}")

        ##### LSTM Model
        model = initialize_model(X_train, y_train, n_unit=n_unit)
        model = compile_model(model, learning_rate=learning_rate)
        model, history = train_model(model,
                                     X_train,
                                     y_train,
                                     validation_split = 0.2,
                                     batch_size = batch_size,
                                     epochs = epochs)

        # Create a figure and axes object with 1 row and 2 columns
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

        # Plot training & validation MAE values
        axes[0].plot(history.history['mae'])
        axes[0].plot(history.history['val_mae'])
        axes[0].set_title('Model MAE')
        axes[0].set_ylabel('MAE')
        axes[0].set_xlabel('Epoch')
        axes[0].legend(['Train', 'Validation'], loc='upper left')

        # Plot training & validation loss values
        axes[1].plot(history.history['loss'])
        axes[1].plot(history.history['val_loss'])
        axes[1].set_title('Model Loss')
        axes[1].set_ylabel('Loss')
        axes[1].set_xlabel('Epoch')
        axes[1].legend(['Train', 'Validation'], loc='upper left')

        # Adjust layout to prevent overlap
        plt.tight_layout()

        # Show the plot
        plt.show()

        res = model.evaluate(X_val, y_val, verbose=0)    # evaluating LSTM (metric)
        mae_lstm = res[1]
        list_of_mae_recurrent_model.append(mae_lstm)
        print(f"MAE LSTM fold n°{fold_id} = {round(mae_lstm, 2)}")

        ##### Comparison LSTM vs Baseline for the current fold
        print(f"improvement over baseline: {round((1 - (mae_lstm/mae_baseline))*100,2)} % \n")

    return list_of_mae_baseline_model, list_of_mae_recurrent_model


In [ ]:
mae_baselines, mae_lstms = cross_validate_baseline_and_lstm()

In [ ]:
print(f"average percentage improvement over baseline = {round(np.mean(1 - (np.array(mae_lstms)/np.array(mae_baselines))),2)*100}%")

# Prediction

In [ ]:
X_train, y_train = get_X_y_seq(train, number_of_sequences=10000, input_length=48, output_length=24, gap_hours=12)

In [ ]:
model = initialize_model(X_train, y_train, n_unit=n_unit)
model = compile_model(model, learning_rate=learning_rate)
model, history = train_model(model,
                                X_train,
                                y_train,
                                validation_split = 0.1,
                                batch_size = batch_size,
                                epochs = epochs)

In [ ]:
from power.params import *
from power.ml_ops.data import get_data_with_cache
from pathlib import Path
import tensorflow as tf

input_pred = '2021-05-08 12:00:00'

query = f"""
    SELECT *
    FROM {GCP_PROJECT}.{BQ_DATASET}.processed_pv
    ORDER BY utc_time
"""

data_processed_cache_path = Path(LOCAL_DATA_PATH).joinpath("processed", f"processed_pv.csv")
data_processed = get_data_with_cache(
    gcp_project=GCP_PROJECT,
    query=query,
    cache_path=data_processed_cache_path,
    data_has_header=True
)

X_pred = data_processed[data_processed['utc_time'] < input_pred][-48:]

In [ ]:
X_pred = X_pred[['electricity']]

In [ ]:
X_pred = X_pred.to_numpy()
X_pred_tf = tf.convert_to_tensor(X_pred)
X_pred_tf = tf.expand_dims(X_pred_tf, axis=0)

In [ ]:
y_pred = model.predict(X_pred_tf)

In [ ]:
y_pred[0]

In [ ]:
plt.plot(y_pred[0])